In [3]:
from datetime import datetime
import google.cloud.aiplatform as aip
from google_cloud_pipeline_components import aiplatform as gcc_aip
import kfp
from kfp import dsl
from kfp.v2 import compiler
from kfp.v2.dsl import component
import json
from kfp.v2.google.client import AIPlatformClient
from kfp.v2.dsl import (Artifact,
                        ClassificationMetrics, 
                        component,
                        Dataset,
                        Input,
                        Metrics,
                        Model,
                        Output
                       )
from typing import NamedTuple

In [14]:
# Pre-Configured Values
Val = !gcloud config list --format 'value(core.project)'
PROJECT_ID = Val[0]
print(f"PROJECT_ID - {PROJECT_ID}")
REGION = "asia-east2"
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
SERVICE_ACCOUNT = "terraform-sa@master-314712.iam.gserviceaccount.com"
BUCKET_NAME = "gs://master_asia_east_2/Vertex_AI/Sentiment_Analysis"
PIPELINE_ROOT = f"{BUCKET_NAME}/Pipeline_Root/Starter"
print(f"PIPELINE_ROOT - {PIPELINE_ROOT}")
PIPELINE_JSON_FILE = "Starter_Analysis.json"
PIPELINE_EXPERIMENT_NAME = "Starter_Scoring_Pipeline"+TIMESTAMP
MODEL_DISPLAY_NAME = "Sentiment_analysis"

PROJECT_ID - master-314712
PIPELINE_ROOT - gs://master_asia_east_2/Vertex_AI/Sentiment_Analysis/Pipeline_Root/Starter


In [15]:
aip.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_NAME)

In [ ]:
@component(base_image="gcr.io/ml-pipeline/google-cloud-pipeline-components:latest",
           packages_to_install = ["pandas"],          
          )
def bq_load(
    train_data: Output[Dataset]
) -> str:
    import pandas
    from google.cloud import bigquery
    client = bigquery.Client(location="US", project='hackteam-mythbusters1')
    
    query = """
    SELECT * FROM `hackteam-mythbusters1.covid_dataset.combined1`
    """
    query_job = client.query(
        query,
        location="US",
    )

    df = query_job.to_dataframe()
    df.to_csv(train_data.path, index=False)
    return(train_data.path.replace("/gcs/", "gs://"))